In [1]:
!wget http://curtis.ml.cmu.edu/datasets/hotpot/hotpot_train_v1.1.json

--2024-10-09 20:12:54--  http://curtis.ml.cmu.edu/datasets/hotpot/hotpot_train_v1.1.json
Resolving curtis.ml.cmu.edu (curtis.ml.cmu.edu)... 128.2.204.193
Connecting to curtis.ml.cmu.edu (curtis.ml.cmu.edu)|128.2.204.193|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 566426227 (540M) [application/json]
Saving to: ‘hotpot_train_v1.1.json’

hotpot_train_v1.1.j 100%[===================>] 540.19M   111MB/s    in 5.0s    

2024-10-09 20:12:59 (108 MB/s) - ‘hotpot_train_v1.1.json’ saved [566426227/566426227]



In [2]:
import json
import pandas as pd
import random

from typing import List, Tuple

In [3]:
def generate_context(input: Tuple[str, List[str]]):
    title, context = input
    return f"{title}\n{''.join(context)}"


def generate_contexts(contexts: List[Tuple[str, List[str]]]):
    context_strings = "\n\n".join([generate_context(context) for context in contexts])
    return f"Contexts:\n\n{context_strings}"


def generate_question(question: str):
    return f"Question: {question}"

In [4]:
df_train_raw = json.load(open("hotpot_train_v1.1.json", "rb"))
random.shuffle(df_train_raw)

In [5]:
df_train = df_train_raw[:-1000]
df_test = df_train_raw[-1000:]

df_train = pd.DataFrame(df_train)
df_test = pd.DataFrame(df_test)

df_train["context"] = df_train["context"].apply(generate_contexts)
df_test["context"] = df_test["context"].apply(generate_contexts)

df_train["question"] = df_train["question"].apply(generate_question)
df_test["question"] = df_test["question"].apply(generate_question)

df_train["source"] = df_train.apply(
    lambda row: f"{row['context']}\n\n{row['question']}", axis=1
)
df_test["source"] = df_test.apply(
    lambda row: f"{row['context']}\n\n{row['question']}", axis=1
)

In [6]:
df_train = df_train.rename(columns={"_id": "id", "answer": "target"})
df_test = df_test.rename(columns={"_id": "id", "answer": "target"})

In [7]:
df_train = df_train[["id", "source", "target"]]
df_test = df_test[["id", "source", "target"]]

In [8]:
df_train.to_csv("train.csv", index=False)
df_test.to_csv("test.csv", index=False)